Preprossin Data

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error,confusion_matrix, ConfusionMatrixDisplay,classification_report, precision_recall_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score,train_test_split, learning_curve, cross_val_predict
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor


In [3]:
df = pd.read_csv('/home/mohammad/code/Reem24Alamri/hospitai/raw_data/healthcare_prediction_dataset.csv')

df.head()


KeyboardInterrupt



In [3]:
df.shape

(55500, 15)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [5]:
df.describe()  # Descriptive statistics for numerical features

,Age,Billing Amount,Room Number
count,55500.000000,55500.000000,55500.000000
mean,51.539459,25539.316097,301.134829
std,19.602454,14211.454431,115.243069
min,13.000000,-2008.492140,101.000000
25%,35.000000,13241.224652,202.000000
50%,52.000000,25538.069376,302.000000
75%,68.000000,37820.508436,401.000000
max,89.000000,52764.276736,500.000000


In [6]:
df.duplicated().sum()

534

In [7]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [8]:
df.isnull().sum().sort_values(ascending=False)

Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64

In [9]:
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54966 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                54966 non-null  object        
 1   Age                 54966 non-null  int64         
 2   Gender              54966 non-null  object        
 3   Blood Type          54966 non-null  object        
 4   Medical Condition   54966 non-null  object        
 5   Date of Admission   54966 non-null  datetime64[ns]
 6   Doctor              54966 non-null  object        
 7   Hospital            54966 non-null  object        
 8   Insurance Provider  54966 non-null  object        
 9   Billing Amount      54966 non-null  float64       
 10  Room Number         54966 non-null  int64         
 11  Admission Type      54966 non-null  object        
 12  Discharge Date      54966 non-null  datetime64[ns]
 13  Medication          54966 non-null  object    

In [10]:
df['StayDuration'] = ((df['Discharge Date'] - df['Date of Admission'])).dt.days
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,StayDuration
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,6
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,15
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,30
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,20


In [11]:
data = df.drop(['Name','Doctor','Hospital','Room Number', 'Date of Admission', 'Discharge Date'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54966 entries, 0 to 55499
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 54966 non-null  int64  
 1   Gender              54966 non-null  object 
 2   Blood Type          54966 non-null  object 
 3   Medical Condition   54966 non-null  object 
 4   Insurance Provider  54966 non-null  object 
 5   Billing Amount      54966 non-null  float64
 6   Admission Type      54966 non-null  object 
 7   Medication          54966 non-null  object 
 8   Test Results        54966 non-null  object 
 9   StayDuration        54966 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 4.6+ MB


In [12]:
data = pd.get_dummies(data, columns=['Gender', 'Blood Type', 'Medical Condition', 'Test Results', 'Admission Type', 'Medication'])


In [13]:
data['Billing Amount'] = data['Billing Amount'].round

In [18]:
data.head()

,Age,Insurance Provider,Billing Amount,StayDuration,Gender_Female,Gender_Male,Blood Type_A+,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,...,Test Results_Inconclusive,Test Results_Normal,Admission Type_Elective,Admission Type_Emergency,Admission Type_Urgent,Medication_Aspirin,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin
0,30,Blue Cross,<bound method Series.round of 0 18856.2...,2,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1,62,Medicare,<bound method Series.round of 0 18856.2...,6,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,76,Aetna,<bound method Series.round of 0 18856.2...,15,1,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
3,28,Medicare,<bound method Series.round of 0 18856.2...,30,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,43,Aetna,<bound method Series.round of 0 18856.2...,20,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [15]:
unique_counts = data.nunique()
print(unique_counts)

Age                               77
Insurance Provider                 5
Billing Amount                     1
StayDuration                      30
Gender_Female                      2
Gender_Male                        2
Blood Type_A+                      2
Blood Type_A-                      2
Blood Type_AB+                     2
Blood Type_AB-                     2
Blood Type_B+                      2
Blood Type_B-                      2
Blood Type_O+                      2
Blood Type_O-                      2
Medical Condition_Arthritis        2
Medical Condition_Asthma           2
Medical Condition_Cancer           2
Medical Condition_Diabetes         2
Medical Condition_Hypertension     2
Medical Condition_Obesity          2
Test Results_Abnormal              2
Test Results_Inconclusive          2
Test Results_Normal                2
Admission Type_Elective            2
Admission Type_Emergency           2
Admission Type_Urgent              2
Medication_Aspirin                 2
M

In [16]:

# # Initialize and train the Random Forest Regressor
# rf = RandomForestRegressor(random_state=42)
# rf.fit(X_train, y_train)
# # Make predictions
# y_pred = rf.predict(X_test)
# y_pred

In [ ]:
model = LinearRegression()


X = df.drop('StayDuration', axis=1)
y = df['StayDuration']

# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = model.predict(X_test)

model.fit(X_train, y_train)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [24]:
from hospital.ml_logic.data import get_data
get_data

ImportError: cannot import name 'get_data' from 'data' (/home/mohammad/code/Reem24Alamri/hospitai/data.py)

In [1]:
import os
import pandas as pd

In [2]:
os.getcwd()

'/home/mohammad/code/Reem24Alamri/hospitai'

In [3]:
csv_path = os.path.join('raw_data')
csv_path

'raw_data'

In [4]:
pd.read_csv(os.path.join(csv_path, 'HDHI Admission data.csv')).head()

,SNO,MRD No.,D.O.A,D.O.D,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1,234735,4/1/2017,4/3/2017,81,M,R,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
1,2,234696,4/1/2017,4/5/2017,65,M,R,E,Apr-17,5,...,0,0,0,0,0,0,0,0,0,0
2,3,234882,4/1/2017,4/3/2017,53,M,U,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
3,4,234635,4/1/2017,4/8/2017,67,F,U,E,Apr-17,8,...,0,0,0,0,0,0,0,0,0,0
4,5,234486,4/1/2017,4/23/2017,60,F,U,E,Apr-17,23,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from hospital.ml_logic.data import get_data_cleaned

df = get_data_cleaned()

df

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,DURATION OF STAY,SMOKING,ALCOHOL,DM,HTN,CAD,...,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,sin_admission,cos_admission
0,81,M,R,E,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
1,65,M,R,E,5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
2,53,M,U,E,3,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
3,67,F,U,E,8,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
4,60,F,U,E,23,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15751,60,F,U,E,9,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15752,86,F,U,O,5,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15754,82,M,U,E,10,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15755,59,F,U,O,4,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01


In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from hospital.ml_logic.data import get_data_cleaned
from hospital.ml_logic.preprocessing import sk_learn_proc
from hospital.ml_logic.model import model_initalize



df = get_data_cleaned()
df
# df_proc = sk_learn_proc(df)
# model_initalize(sk_learn_proc(X, y))

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,DURATION OF STAY,SMOKING,ALCOHOL,DM,HTN,CAD,...,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,sin_admission,cos_admission
0,81,M,R,E,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
1,65,M,R,E,5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
2,53,M,U,E,3,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
3,67,F,U,E,8,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
4,60,F,U,E,23,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15751,60,F,U,E,9,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15752,86,F,U,O,5,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15754,82,M,U,E,10,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01
15755,59,F,U,O,4,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0.866025,5.000000e-01


In [2]:
X, y = sk_learn_proc(df)

In [3]:
X

,AGE,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,EF,GENDER,RURAL,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1.117647,-0.967742,1.24,0.964602,-0.678571,0.000000,-0.017241,-0.33845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.176471,0.387097,-0.18,-0.699115,-0.297619,-0.592593,-0.017241,-0.05845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.529412,-0.612903,0.96,0.893805,0.595238,2.185185,2.396552,0.00000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.096774,0.00,0.513274,-0.083333,-0.259259,-0.534483,-0.05845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.117647,0.354839,-0.16,-1.787611,0.083333,0.777778,0.586207,-1.09845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,-0.117647,-1.419355,3.02,-1.460177,1.011905,-0.592593,-0.706897,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13751,1.411765,-1.193548,0.76,1.176991,-0.071429,0.851852,0.844828,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13752,1.176471,-1.032258,0.36,1.274336,0.869048,1.222222,1.706897,-0.45845,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13753,-0.176471,0.193548,0.52,1.796460,0.190476,-0.185185,-0.189655,0.66155,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y

0         3
1         5
2         3
3         8
4        23
         ..
13750     9
13751     5
13752    10
13753     4
13754     4
Name: DURATION OF STAY, Length: 13755, dtype: int64

In [5]:
model = model_initalize()

model

KNeighborsRegressor(metric='manhattan', n_neighbors=11, weights='distance')

In [6]:
from hospital.ml_logic.model import train_model, evaluate_model, save_model

model = model_initalize()
model = train_model(model, X, y)

evaluate_model(model, X, y)

Mean Absolute Error (MAE): 0.5087069679069858


In [9]:
save_model(model, 'knn_model.pkl')

Model has been saved as knn_model.pkl


In [10]:
import pickle

# Load the saved model from the pickle file
with open('knn_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Now you can use `loaded_model` to make predictions just like the original model


In [12]:
import pickle
from sklearn.metrics import mean_absolute_error

# Step 1: Load the model
with open('knn_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Step 2: Use the loaded model for predictions
y_pred = loaded_model.predict(X_test)

# Step 3: Evaluate the loaded model
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE) for the loaded model: {mae}')

NameError: name 'X_test' is not defined